# Librerias


In [361]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats


In [362]:
# Ruta del archivo transplante consolidado (sin procesar)
path_transplante = r'C:\Users\SSN0609\OneDrive - Stanley Black & Decker\Sebastian\lina\Data\Raw\Trasplante_hepático_consolidado.xlsx'
# De este dataframe se extraerán las variables que se utilizarán en el reporte de Power BI
df_transplante_base = pd.read_excel(path_transplante,sheet_name='Base',dtype=str)
#De este dataframe se extraerán los códigos de las variables que se utilizarán en el reporte de Power BI
df_transplante_diccionario=pd.read_excel(path_transplante,sheet_name='Codificacion',dtype=str,skiprows=1)
df_transplante_diccionario=df_transplante_diccionario.apply(lambda col: col.astype(str).str.strip())
df_transplante_diccionario.rename(columns={'INTRUCCIÓN ':'ROW_ID'}, inplace=True)


# Variables BI

Se crea una lista con las variables, con las cuales se realizara el reporte Power Bi, y por lo tanto las variables a las cuales se les realizara un proceso de limpieza

In [363]:
lst_diccionario=['DIAGNÓSTICO ENFERMEDAD HEPÁTICA #1 QUE LLEVA AL TRASPLANTE',
                 'SI TUVO COMPLICACIONES EN EL POP',
                 'DEFINE EL TIPO DE LA COMPLICACIÓN PRINCIPAL ',
                 'ASEGURADORA',
                 'LETRA DEL CHILD PUGH PRETRASPLANTE(No aplica si no tiene cirrosis)',
                 'DEFINE EL CIRUJANO QUE RESCATÓ',
                 'CIRUJANO AYUDANTE DEL TX',
                 'DEFINE EL AGENTE INFECCIOSO DEL PRIMER TIPO DE INFECCIÓN',
                 'CAUSA DE MUERTE DEL PACIENTE TRASPLANTADO',
                 'TIPO DE NEOPLASIA POST TX']

lst_replace=["nv","****","8888","7777","9999","999","", "nan"]

lst_drop=['INTRUCCION',
          'Causa_1_Tx_Hepático_',
          'fk_tipo_complicacion',
          'EPS']

lstcolumnasBI =['Fecha_Tx',
                'Fecha_Control/Muerte',
'Sexo_Receptor',#ok
'Edad_Al_Tx',#ok
'Causa_1_Tx_Hepático_',#ok merge: DIAGNÓSTICO ENFERMEDAD HEPÁTICA #1 QUE LLEVA AL TRASPLANTE
'¿Complicación?',#ok merge:SI TUVO COMPLICACIONES EN EL POP
'Tipo_De_Complicación_1',#ok merge : DEFINE EL TIPO DE LA COMPLICACIÓN PRINCIPAL 
'EPS',#ok merge: ASEGURADORA
'Tiempo_En_Lista_Meses',# replace ['1507.0','1510.1','1510.8','1524.7','1524.8','1524.8']'
'Código de anonimización ',
'Año_Tx',#ok
'Grupo_Sanguíneo_Receptor',#quitar los numeros de las letras, replace (0,9999,blank)
'RH_Receptor',#replace +- por nhd
'Child_Pugh_Letra',#replace (0,9999,blank,1,2,3)
'MELD_Score',#revisar lina
'Peso_Pre_Tx',#ok
'BMI',#replace(9999,%peso)
'Falla_Cardiaca_Pre_Tx',#replace(9999,blank),pericarditis=1
'Enfermedad_Coronaria_Pre_Tx',#replace(9999,blank),angina=1
'Infarto_Cardiaco_Pre_Tx',# replace(9999,blank),infarto=1
'Diabetes_Mellitus_Pre_Tx',#replace(9999,blank),2=1,o=0
'Hipertensión_Arterial_Pre_Tx',#replace(9999,blank),o=0
'Antecedente_De_Tabaquismo',#replace(9999,blank),o=0
'Antecedente_De_Alcoholismo',#replace(9999,blank),o=0
'LDL_Pre_Tx',#ok
'HDL_Pre_Tx',#ok
'Triglicéridos_Pre_Tx',#replace(9999,blank)
'Colesterol_TotalPre_Tx',#replace(9999,blank,nv)
'#_Unidades_Plaquetas_Tx#1',#replace(9999,blank,nv)
'#_Unidades_Plasma_Fresco_Congelado_Tx#1',#replace(9999,blank,nv)
'#_Unidades_Glóbulos_Rojos_Tx#1',#replace(9999,blank,nv)
'Cirujano_Principal_Tx#1',#DEFINE EL CIRUJANO QUE RESCATÓ
'Cirujano_Ayudante_Tx#1',#ok
'Tiempo_Quirúrgico_Tx#1',#ok
'Días_Tratamiento_Antibiótico_Previo_A_Tx#1',
'Antibiótico_1__Tx#1',#replace(9999,blank)
'Infección_POP',#replace(9999,blank)
'Germen_1',#replace(ecoli,e.coli) DEFINE EL AGENTE INFECCIOSO DEL PRIMER TIPO DE INFECCIÓN
'Retrasplante',#ok
'Días_En_UCI',#ok
'Días_Totales_Intrahospitalarios',#replace numeros negativos por nhd
'Sobrevida_PeriQx_30_días_POP',#ok
'Vivo_Hoy',#ok
'Causa_De_Muerte',# replace ( 9999,****CAUSA DE MUERTE DEL PACIENTE TRASPLANTADO
'Neoplasia_PostTx',#replace(8888,7777) TIPO DE NEOPLASIA POST TX
'Cirujano_Rescate_1_Tx#1',
'Tiempo_Isquemia_Fría_Tx#1',
'Tiempo_Isquemia_Caliente_Tx#1',
'Profilaxis_Antitrombótica',
'Rechazo_Agudo',
'Rechazo_Crónico',
'Falla_Cardiaca_POP',
'Arritmia_POP',
'Infarto_POP',
'Fast_Track',
'Edad_Donante_Tx#1',
'Sexo_Donante_Tx#1',
'Causa_Muerte_Donante_Tx#1',
'CMV_Donante_Tx#1',
'#_De_Complicaciones',
'Composición_Corporal'
]

In [364]:
# Se crea una copia del dataframe base y se filtran las columnas necesarias para el reporte BI
df_bi = df_transplante_base.copy()
df_bi = df_bi[lstcolumnasBI]

# Tratamiento Variables

In [365]:
def limpiar_y_reemplazar(series, replace_list):
    # Asegura que la entrada sea string, limpia, convierte a mayúsculas
    series = series.astype(str).str.strip().str.upper()
    # Realiza el reemplazo
    return series.replace(replace_list, "NHD")

# Aplicación a df_bi
df_bi = df_bi.apply(lambda x: limpiar_y_reemplazar(x, lst_replace), axis=0)

# Aplicación a df_transplante_diccionario
df_transplante_diccionario = df_transplante_diccionario.apply(lambda x: limpiar_y_reemplazar(x, lst_replace), axis=0)

## Tratamiento Especifico por Variable

Se realiza inputacion de datos, para un listado de variables que tienen particularidades unicas.

### Causa 1 Tx Hepatico

In [366]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','DIAGNÓSTICO ENFERMEDAD HEPÁTICA #1 QUE LLEVA AL TRASPLANTE']],
    how="left",
    left_on='Causa_1_Tx_Hepático_',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Causa_1_Tx_Hepático_'], inplace=True)

### ¿Complicacion?

In [367]:
#¿Complicación?, esta variable se trata de una variable binaria, por lo tanto se le asigna el valor 0 si no hay complicación y 1 si hay complicación
# Se eliminan espacios, se convierten a mayúsculas y se reemplazan los valores
df_bi['¿Complicación?'] = df_bi['¿Complicación?'].str.strip().str.upper().str.replace(' ', '')
df_bi['¿Complicación?'] = df_bi['¿Complicación?'].replace(['NO','N0','NINGUNA'], '0')
df_bi['¿Complicación?'] = df_bi['¿Complicación?'].apply(
    lambda x: "1" if (not pd.isna(x)) and (x not in ['0', "NHD"]) else x
)

### Tipo Complicación
Esta variable contiene los siguiente problemas
* Contiene el id de la complicación pero no su nombre, necesitamos asignar el nombre
* Algunos registros contienen mas de un tipo de complicación, nos quedaremos solo con la primera complicación( dado que los casos con varias son muy pocos)
* Algunos registros no indican el tipo de complicación

In [368]:
# Creamos una nueva columna 'Tipo_complicacion' para almacenar el ID tipo de complicación, limpio
df_bi['Tipo_complicacion'] = ""
for i, fila in df_bi['Tipo_De_Complicación_1'].items():
    fila = str(fila).strip()
    # Tiene coma o punto
    if ',' in fila:
        valor = fila.split(',')[0]
    elif '.' in fila:
        valor = fila.split('.')[0]
    # Más de 3 caracteres
    elif len(fila) > 3:
        valor = "NHD"
    # Todo lo demás
    else:
        valor = fila
    df_bi.loc[i, 'Tipo_complicacion'] = valor
    
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','DEFINE EL TIPO DE LA COMPLICACIÓN PRINCIPAL ']],
    how="left",
    left_on='Tipo_complicacion',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Tipo_complicacion'], inplace=True)


### EPS


In [369]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','ASEGURADORA']],
    how="left",
    left_on="EPS",
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','EPS'], inplace=True)


### Tiempo lista en Meses


In [370]:
df_bi['Tiempo_En_Lista_Meses'] = pd.to_numeric(
    df_bi['Tiempo_En_Lista_Meses'], 
    errors='coerce' #
)
df_bi['Tiempo_En_Lista_Meses']=df_bi['Tiempo_En_Lista_Meses'].astype(float)
df_bi['Tiempo_En_Lista_Meses'] = np.where(
    # Condición: si el valor es menor que 0 O mayor que 15000
    (df_bi['Tiempo_En_Lista_Meses'] < 0) | (df_bi['Tiempo_En_Lista_Meses'] > 15000) ,
    
    # Valor si la condición es VERDADERA: reemplazar con NaN
    np.nan,
    
    # Valor si la condición es FALSA: mantener el valor original
    df_bi['Tiempo_En_Lista_Meses']
)



### Grupo Sanguineo

In [371]:
df_bi['Grupo_Sanguíneo_Receptor'] = df_bi['Grupo_Sanguíneo_Receptor'].str[:2]

# 2. Definir las condiciones y las elecciones
condiciones = [
    df_bi['Grupo_Sanguíneo_Receptor'].str.startswith("AB", na=False),
    df_bi['Grupo_Sanguíneo_Receptor'].str.startswith("B", na=False),
    df_bi['Grupo_Sanguíneo_Receptor'].str.startswith("A", na=False),
    df_bi['Grupo_Sanguíneo_Receptor'].str.startswith("O", na=False)
]

# Las elecciones (resultados) correspondientes a cada condición
elecciones = [
    "AB",
    "B",
    "A",
    "O"
]

# 3. Aplicar np.select
# 'NHD' es el valor por defecto si ninguna condición es verdadera
df_bi['Grupo_Sanguíneo_Receptor'] = np.select(
    condiciones,
    elecciones,
    default="NHD"
)

### RH Receptor

In [372]:
lst_rh=['(+)(-)','9999','NAN']
df_bi['RH_Receptor'] = df_bi['RH_Receptor'].replace(lst_rh, 'NHD')

### Child Pugh


In [373]:
condiciones = [
    df_bi['Child_Pugh_Letra'].str.startswith("1", na=False),
    df_bi['Child_Pugh_Letra'].str.startswith("2", na=False),
    df_bi['Child_Pugh_Letra'].str.startswith("3", na=False),
    df_bi['Child_Pugh_Letra'].str.startswith("0", na=False)

]
elecciones = [
    "A",
    "B",
    "C",
    "No Aplica"
]

df_bi['Child_Pugh_Letra'] = np.select(
    condiciones,
    elecciones,
    default="NHD"
)


### Meld Score


In [374]:
df_bi['MELD_Score']=df_bi['MELD_Score'].replace(['9999','NAN',""], 'NHD')

### BMI

In [375]:
df_bi['BMI']=pd.to_numeric(
    df_bi['BMI'], 
    errors='coerce' #
)
df_bi['BMI']=df_bi['BMI'].astype(float)
df_bi['BMI']=df_bi['BMI'].replace(['9999'], '0')

# Composicion corporal

In [376]:
def clasificar_bmi(bmi):
    if pd.isna(bmi):
        return np.nan
    elif bmi < 18.5:
        return 0
    elif bmi < 25:
        return 1
    elif bmi < 30:
        return 2
    elif bmi < 35:
        return 3
    elif bmi < 40:
        return 4
    else:
        return 5
mask_restaurar_nhd = df_bi['Composición_Corporal'].isna() & df_bi['BMI'].isna()
df_bi.loc[mask_restaurar_nhd, 'Composición_Corporal'] = 'NHD'

mask_imputa = df_bi['Composición_Corporal'].isin(['NHD', '9999', 9999]) & df_bi['BMI'].notna()
df_bi.loc[mask_imputa, 'Composición_Corporal'] = (
    df_bi.loc[mask_imputa, 'BMI'].apply(clasificar_bmi)
)

### Falla Cardiaca pre tx

In [377]:
df_bi['Falla_Cardiaca_Pre_Tx']=df_bi['Falla_Cardiaca_Pre_Tx'].replace(['9999','NAN',""], 'NHD')
df_bi['Falla_Cardiaca_Pre_Tx']=df_bi['Falla_Cardiaca_Pre_Tx'].replace(['PERICARDITIS'], 'NHD')


### Emfermedad Coronaria

In [378]:
df_bi['Enfermedad_Coronaria_Pre_Tx']=df_bi['Enfermedad_Coronaria_Pre_Tx'].replace(['9999','NAN',""], 'NHD')
df_bi['Enfermedad_Coronaria_Pre_Tx'] = np.where(
    # Condición: si el valor contiene la subcadena "ANGINA" (ignora NaN con na=False)
    df_bi['Enfermedad_Coronaria_Pre_Tx'].str.contains("ANGINA", na=False),
    
    # Valor si la condición es VERDADERA: 1
    '1',
    
    # Valor si la condición es FALSA: Mantener el valor original
    df_bi['Enfermedad_Coronaria_Pre_Tx']
)

### Infarto Cardiaco

In [379]:
df_bi['Infarto_Cardiaco_Pre_Tx']=df_bi['Infarto_Cardiaco_Pre_Tx'].replace(['9999','NAN',""], 'NHD')
df_bi['Infarto_Cardiaco_Pre_Tx']=df_bi['Infarto_Cardiaco_Pre_Tx'].replace(['BLOQUEO AV CON MARCAPASOS'], '1')


### Ajuste multiples variables
Se crea una lista de variables que poseen problematicas similares de informacion

In [380]:
lst=['Diabetes_Mellitus_Pre_Tx',
     'Hipertensión_Arterial_Pre_Tx',
'Antecedente_De_Tabaquismo',
'Antecedente_De_Alcoholismo',
'Triglicéridos_Pre_Tx',#replace(9999,blank)
'Colesterol_TotalPre_Tx',#replace(9999,blank,nv)
'#_Unidades_Plaquetas_Tx#1',#replace(9999,blank,nv)
'#_Unidades_Plasma_Fresco_Congelado_Tx#1',#replace(9999,blank,nv)
'#_Unidades_Glóbulos_Rojos_Tx#1',#replace(9999,blank,nv)
'Antibiótico_1__Tx#1',#replace(9999,blank)
'Infección_POP',#replace(9999,blank)
'Causa_De_Muerte',# replace ( 9999,****CAUSA DE MUERTE DEL PACIENTE TRASPLANTADO
'Neoplasia_PostTx'#replace(8888,7777) TIPO DE NEOPLASIA POST TX
]

for col in df_bi[lst]:
    df_bi[col]=df_bi[col].str.strip().str.upper()   # quita espacios
    df_bi[col] = df_bi[col].replace(['9999','NAN',"",'NV','8888','7777','****'], 'NHD')

 

### Cirujano principal

In [381]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','CIRUJANO PRINCIPAL DEL TX']],
    how="left",
    left_on='Cirujano_Principal_Tx#1',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Cirujano_Principal_Tx#1'], inplace=True)

### Cirujano auxiliar

In [382]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','CIRUJANO AYUDANTE DEL TX']],
    how="left",
    left_on='Cirujano_Ayudante_Tx#1',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Cirujano_Ayudante_Tx#1'], inplace=True)

### Días tratamiento antibiotico

In [383]:
df_bi['Días_Tratamiento_Antibiótico_Previo_A_Tx#1']= df_bi['Días_Tratamiento_Antibiótico_Previo_A_Tx#1'].replace(['9999','NAN',"","-"], '')

### Germen 1

In [384]:
condiciones = [
    df_bi['Germen_1'].str.contains("E.COLI", na=False),
    df_bi['Germen_1'].str.contains("ECOLI", na=False),
    df_bi['Germen_1'].str.contains("CANDIDA", na=False),
    df_bi['Germen_1'].str.contains("ENTEROBACTER", na=False),
    df_bi['Germen_1'].str.contains("KPC", na=False),
    df_bi['Germen_1'].str.contains("CAPITIS", na=False)
    ]

elecciones = [
    "7",
    "7",
    "15",
    "27",
    "28",
    "19"]
df_bi['Germen_1'] = np.select(
    condiciones,
    elecciones,
    default=df_bi['Germen_1']
)

df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','DEFINE EL AGENTE INFECCIOSO DEL PRIMER TIPO DE INFECCIÓN']],
    how="left",
    left_on='Germen_1',
    right_on='ROW_ID')

df_bi.drop(columns=['ROW_ID','Germen_1'], inplace=True)

### Días totales intrahospitalarios

In [385]:
df_bi['Días_Totales_Intrahospitalarios']=pd.to_numeric(
    df_bi['Días_Totales_Intrahospitalarios'],   
    errors='coerce' #
)



df_bi['Días_Totales_Intrahospitalarios']=df_bi['Días_Totales_Intrahospitalarios'].astype(float)
condiciones = [
    df_bi['Días_Totales_Intrahospitalarios'] < 0,
    df_bi['Días_Totales_Intrahospitalarios'] > 9000,
]
elecciones = [
    np.nan,
    np.nan]
df_bi['Días_Totales_Intrahospitalarios'] = np.select(
    condiciones,
    elecciones,
    default=df_bi['Días_Totales_Intrahospitalarios']
)

### Causa de muerte

In [386]:
condiciones = [
    df_bi['Causa_De_Muerte'].str.contains("FALLA MULTI", na=False),
    df_bi['Causa_De_Muerte'].str.contains("SEPSIS", na=False)
]

elecciones = [
    "12",
    "2"
]
df_bi['Causa_De_Muerte'] = np.select(
    condiciones,
    elecciones,
    default=df_bi['Causa_De_Muerte']
)

df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','CAUSA DE MUERTE DEL PACIENTE TRASPLANTADO']],
    how="left",
    left_on='Causa_De_Muerte',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Causa_De_Muerte'], inplace=True)

lst_causas_muerte = df_transplante_diccionario['CAUSA DE MUERTE DEL PACIENTE TRASPLANTADO'].unique().tolist()
condiciones = [
    df_bi['CAUSA DE MUERTE DEL PACIENTE TRASPLANTADO'].isin(lst_causas_muerte)]
elecciones = [
    df_bi['CAUSA DE MUERTE DEL PACIENTE TRASPLANTADO']]
df_bi['CAUSA DE MUERTE DEL PACIENTE TRASPLANTADO'] = np.select(
    condiciones,    
    elecciones,
    default="Otra")

    

### Neoplasia post tx

In [387]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','TIPO DE NEOPLASIA POST TX']],
    how="left",
    left_on='Neoplasia_PostTx',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Neoplasia_PostTx'], inplace=True)

# Cirujano_Rescate_1_Tx#1


In [388]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','DEFINE EL CIRUJANO QUE RESCATÓ']],
    how="left",
    left_on='Cirujano_Rescate_1_Tx#1',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Cirujano_Rescate_1_Tx#1'], inplace=True)

In [389]:
lst_int = ['Tiempo_Isquemia_Fría_Tx#1', 'Tiempo_Isquemia_Caliente_Tx#1']

for col in lst_int:
    # 1. Coerción a numérico (convierte valores sucios a NaN)
    df_bi[col] = pd.to_numeric(df_bi[col], errors='coerce') 
    
    # 2. Rellenar los valores NaN con un valor escalar, como 0. 
    # (Si se requiere otro tipo de imputación, se sustituye el 0 aquí).
    df_bi[col] = df_bi[col].fillna(0)
    
    # 3. Conversión a tipo entero (funciona porque ya no hay NaN)
    df_bi[col] = df_bi[col].astype(int)

# Profilaxis_Antitrombótica


In [390]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','DEFINE SI EL PACIENTE REQUIRIÓ PROFILAXIS ANTITROMBÓTICA']],
    how="left",
    left_on='Profilaxis_Antitrombótica',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','DEFINE SI EL PACIENTE REQUIRIÓ PROFILAXIS ANTITROMBÓTICA'], inplace=True)

# Rechazo_Agudo


In [391]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','SI SE DIAGNOSTICÓ RECHAZO AGUDO']],
    how="left",
    left_on='Rechazo_Agudo',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Rechazo_Agudo'], inplace=True)

# Rechazo_Crónico


In [392]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','SI SE DIAGNOSTICÓ RECHAZO CRÓNICO']],
    how="left",
    left_on='Rechazo_Crónico',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','SI SE DIAGNOSTICÓ RECHAZO CRÓNICO'], inplace=True)

# Falla_Cardiaca_POP


In [393]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','SI DESARROLLO FALLA CARDIACA POST TX']],
    how="left",
    left_on='Falla_Cardiaca_POP',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID'], inplace=True)

In [394]:
df_bi['Arritmia_POP']

0      NHD
1      NHD
2      NHD
3      NHD
4      NHD
      ... 
731      0
732      0
733    NAN
734    NAN
735    NAN
Name: Arritmia_POP, Length: 736, dtype: object

# Arritmia_POP


In [395]:
df_bi['Arritmia_POP'].unique()

array(['NHD', '0', '1', '3', 'NAN',
       '4 ( TAQUICARDIA CON BIGEMINISMO, FA, PARO)', 'PARO',
       'TAQUICARDIA SINUSAL', 'TAQUIARDIA SINUSAL',
       'TAQUICARDIA SINUSAL, BRADICARDIA, PARO, AESP, FIBRILACION VENTRICULAR',
       'TAQUICARDIA SINUSAL,',
       'TAQUICARDIA SINUSAL, TAQUICARDIA SURPAVENTRICULAR.AESP, REANIMAICON, TAQUICARDIA VENTRICULAR, FIBRILACION VENTRICULAR. MUERTE',
       'AMBIOS DINAMICOS EN EL VISOSCOPIO: \nARRITMICA, TAQUI-BRADICARDICO, \nELEVACION DEL ST Y PROL\nONGACION DEL QRS,. /  FIBRILACION AURICULAR OCN REPSUESTA VENTRICULAR 112-120, NO COLAPSANTE/ EPISODIOS DE PAUSA SINUSAL EXTREMA / ASISTOLIA REFRACTARIA',
       'BRADICARDIA SINUSAL-ENSANCHAMIENTO QRS/ TAQUICARDIA DUPRAVENTIRCULAR INICIA FIBRILACIN AURICULAR CON RESPUESTA VENTRICULAR RAPIDA',
       'TAQUIARDICA SINUSAL', '2',
       'TAQUICARDIA SINUSAL/ FIBRILACION VENTRICULAR CON RTS VENT CONTROLADA/ EXTRASISTOLES SUPREAVENT OCASIONALES/TAQUICARDIA SUPRAVENTRICULAR CON RESPUESTA VENTRICULAR A

In [396]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','SI DESARROLLO ARRITMIAS POST TX']],
    how="left",
    left_on='Arritmia_POP',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID'], inplace=True)

In [397]:
df_bi['Arritmia_POP'].unique()

array(['NHD', '0', '1', '3', 'NAN',
       '4 ( TAQUICARDIA CON BIGEMINISMO, FA, PARO)', 'PARO',
       'TAQUICARDIA SINUSAL', 'TAQUIARDIA SINUSAL',
       'TAQUICARDIA SINUSAL, BRADICARDIA, PARO, AESP, FIBRILACION VENTRICULAR',
       'TAQUICARDIA SINUSAL,',
       'TAQUICARDIA SINUSAL, TAQUICARDIA SURPAVENTRICULAR.AESP, REANIMAICON, TAQUICARDIA VENTRICULAR, FIBRILACION VENTRICULAR. MUERTE',
       'AMBIOS DINAMICOS EN EL VISOSCOPIO: \nARRITMICA, TAQUI-BRADICARDICO, \nELEVACION DEL ST Y PROL\nONGACION DEL QRS,. /  FIBRILACION AURICULAR OCN REPSUESTA VENTRICULAR 112-120, NO COLAPSANTE/ EPISODIOS DE PAUSA SINUSAL EXTREMA / ASISTOLIA REFRACTARIA',
       'BRADICARDIA SINUSAL-ENSANCHAMIENTO QRS/ TAQUICARDIA DUPRAVENTIRCULAR INICIA FIBRILACIN AURICULAR CON RESPUESTA VENTRICULAR RAPIDA',
       'TAQUIARDICA SINUSAL', '2',
       'TAQUICARDIA SINUSAL/ FIBRILACION VENTRICULAR CON RTS VENT CONTROLADA/ EXTRASISTOLES SUPREAVENT OCASIONALES/TAQUICARDIA SUPRAVENTRICULAR CON RESPUESTA VENTRICULAR A

# Infarto_POP


In [305]:
df_bi['Infarto_POP'].unique()

array(['NHD', '0', 'NAN', 'P', '1', '2', '2 (2 INFARTOS)'], dtype=object)

In [306]:
df_bi['Infarto_POP'].replace(['NAN', 'P'],"NHD",inplace=True)
df_bi['Infarto_POP'].replace('2 (2 INFARTOS)',"2",inplace=True)

df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','SI DESARROLLÓ INFARTO MIOCÁRDICA POST TX']],
    how="left",
    left_on='Infarto_POP',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID'], inplace=True)

C:\Users\SSN0609\AppData\Local\Temp\ipykernel_22532\4020336914.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_bi['Infarto_POP'].replace(['NAN', 'P'],"NHD",inplace=True)


# Fast_Track


In [307]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','DEFINE SI PASÓ A PISO DESDE SALAS SIN PASAR POR UCI (FAST TRACK)']],
    how="left",
    left_on='Fast_Track',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Fast_Track'], inplace=True)

# Edad_Donante_Tx#1


In [308]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','Edad_Donante1']],
    how="left",
    left_on='Edad_Donante_Tx#1',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Edad_Donante_Tx#1'], inplace=True)

# Sexo_Donante_Tx#1


In [309]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','SEXO DEL DONANTE ']],
    how="left",
    left_on='Sexo_Donante_Tx#1',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','SEXO DEL DONANTE '], inplace=True)

# Causa_Muerte_Donante_Tx#1


In [310]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','CAUSA DE LA MUERTE CEREBRAL DEL DONANTE']],
    how="left",
    left_on='Causa_Muerte_Donante_Tx#1',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','Causa_Muerte_Donante_Tx#1'], inplace=True)

# CMV_Donante_Tx#1


In [311]:
df_bi=pd.merge(
    df_bi,
    df_transplante_diccionario[['ROW_ID','ESTADO IGG DEL DONANTE']],
    how="left",
    left_on='CMV_Donante_Tx#1',
    right_on='ROW_ID')
df_bi.drop(columns=['ROW_ID','CMV_Donante_Tx#1'], inplace=True)

# Complicaciones

In [312]:
df_bi['Arritmia_pop_limpia'] = df_bi['Arritmia_POP'].astype(str).str.upper().str.strip()

def clasificar_arritmia(texto):
    if texto in ['0', 'NO', 'N/A', 'NINGUNA']:
        return 0
    # Bradiarritmias
    elif any(x in texto for x in ['BRADI', 'ASISTOLIA', 'RITMO DE ESCAPE', 'AESP', 'PARO']):
        return 3
    # Taquiarritmias ventriculares
    elif any(x in texto for x in ['VENTRICULAR', 'FV', 'FIBRILACION VENTRICULAR', 'BIGEMINISMO']):
        return 2
    # Taquiarritmias supraventriculares
    elif any(x in texto for x in ['SUPRAVENT', 'FA ', 'FIBRILACION AURICULAR', 'TAQUICARDIA SINUSAL']):
        return 1
    # Casos dudosos o combinaciones múltiples
    elif any(c in texto for c in ['+', '/', 'Y']):
        # Si menciona taqui + bradi, podría marcarse como 1 (predomina taqui) o np.nan
        return np.nan
    else:
        return np.nan

df_bi['arritmia_pop_cod'] = df_bi['Arritmia_pop_limpia'].apply(clasificar_arritmia)



In [313]:
df_bi['Arritmia_POP_si_no'] = np.where(
    df_bi['arritmia_pop_cod'] > 0,
    1,
    0
)

In [314]:
df_bi['#_De_Complicaciones'] = df_bi['#_De_Complicaciones'].replace(['NHD', 'NAN'], np.nan)
df_bi['#_De_Complicaciones'] = pd.to_numeric(
    df_bi['#_De_Complicaciones'],
    errors='coerce' # Los strings que no se pudieron convertir (si quedan) se vuelven NaN
)
df_bi['#_De_Complicaciones'] = df_bi['#_De_Complicaciones'].astype('Int64')

In [315]:
# Si el tipo de complicación es disinto de 0, se ausme que hubo una complicación
# Aquí se asume que el tipo de complicación ya se limpio previamente
df_bi.loc[
    (df_bi["Tipo_De_Complicación_1"].notna())
    &(df_bi["Tipo_De_Complicación_1"]!=0)
    & ((df_bi["¿Complicación?"]==0) | 
       (df_bi["¿Complicación?"].isna())),
       '¿Complicación?'] = 1

# Si hubo infección posoperatoria, se asume que hubo complicación 
df_bi.loc[
    (df_bi["Infección_POP"]==1)
    & ((df_bi["¿Complicación?"]==0) | 
       (df_bi["¿Complicación?"].isna())),
       '¿Complicación?'] = 1

# Si hubo arritmia posoperatoria, se asume que hubo complicación.
# Se asume que existe la columna binaria de arritmia luego de la limpieza
df_bi.loc[
    (df_bi["Arritmia_POP_si_no"]==1)
    & ((df_bi["¿Complicación?"]==0) | 
       (df_bi["¿Complicación?"].isna())),
       '¿Complicación?'] = 1

# Si hubo falla cardiaca, se asume que hubo complicación.
df_bi.loc[
    (df_bi["Falla_Cardiaca_POP"]==1)
    & ((df_bi["¿Complicación?"]==0) | 
       (df_bi["¿Complicación?"].isna())),
       '¿Complicación?'] = 1

# Si hubo infarto posoperatorio, se asume que hubo complicación.
df_bi.loc[
    (df_bi["Infarto_POP"]==1)
    & ((df_bi["¿Complicación?"]==0) | 
       (df_bi["¿Complicación?"].isna())),
       '¿Complicación?'] = 1

# Si hay al menos una complicación, se asume que hubo complicaciones
df_bi.loc[
    (df_bi["#_De_Complicaciones"] > 0)
    & (df_bi["¿Complicación?"] == 0),
    "¿Complicación?"] = 0

In [316]:
df_bi['¿Complicación?'].value_counts()

¿Complicación?
0      363
1      339
NHD     34
Name: count, dtype: int64

In [317]:
for col in df_bi.columns:
    print(f'valores unicos para la columna {col}\n')
    print(f'{df_bi[col].unique()}')
          

valores unicos para la columna Fecha_Tx

['1988-06-02 00:00:00' '1989-01-02 00:00:00' '1989-10-21 00:00:00'
 '1990-09-02 00:00:00' '1990-09-18 00:00:00' '1991-03-12 00:00:00'
 '1991-04-07 00:00:00' '1994-03-12 00:00:00' '1994-05-20 00:00:00'
 '1997-03-02 00:00:00' '1998-01-31 00:00:00' '1998-03-03 00:00:00'
 '1998-03-12 00:00:00' '1998-03-30 00:00:00' '1998-04-08 00:00:00'
 '1998-06-06 00:00:00' '1998-07-06 00:00:00' '1998-09-04 00:00:00'
 '1998-09-12 00:00:00' '1998-09-13 00:00:00' '1998-09-25 00:00:00'
 '1998-09-28 00:00:00' '1998-09-30 00:00:00' '1998-10-12 00:00:00'
 '1998-10-18 00:00:00' '1998-10-20 00:00:00' '1998-10-27 00:00:00'
 '1998-12-18 00:00:00' '1998-12-20 00:00:00' '1999-01-04 00:00:00'
 '1999-05-06 00:00:00' '2002-11-11 00:00:00' '2002-11-12 00:00:00'
 '2002-11-28 00:00:00' '2002-12-02 00:00:00' '2003-02-02 00:00:00'
 '2003-03-17 00:00:00' '2003-06-05 00:00:00' '2003-08-08 00:00:00'
 '2003-10-09 00:00:00' '2003-11-24 00:00:00' '2003-11-27 00:00:00'
 '2004-01-02 00:00:00

In [318]:
lst_patterns = ['nan'] 
pattern = '|'.join(lst_patterns)
for col in df_bi.columns:
    df_bi[col] = df_bi[col].astype(str).str.lower().str.replace(pattern, 'NHD', regex=True)  
    df_bi[col]=df_bi[col].str.upper()  
df_bi.fillna("", inplace=True)

In [319]:
df_bi['Child_Pugh_Letra'].unique()

array(['NHD', 'NO APLICA', 'B', 'C', 'A'], dtype=object)

# Analisis de Kaplan Meir

In [320]:
# 1. Asegurar el formato de fecha
df_bi['Fecha_Tx'] = pd.to_datetime(df_bi['Fecha_Tx'], errors='coerce')
df_bi['Fecha_Control/Muerte'] = pd.to_datetime(df_bi['Fecha_Control/Muerte'], errors='coerce')

# Calcular la diferencia de tiempo
diferencia_tiempo = df_bi['Fecha_Control/Muerte'] - df_bi['Fecha_Tx']

# Definir los Timedeltas correctos (365.25 días por año)
un_anio = pd.Timedelta(days=365.25 * 1)
tres_anios = pd.Timedelta(days=365.25 * 3)
cinco_anios = pd.Timedelta(days=365.25 * 5)

# Asumimos que '0' significa Muerte y '1' significa Vivo. Si es al revés, ajusta '0'/'1'.
es_muerte = (df_bi['Vivo_Hoy'] == '0')
# Los valores nulos de fecha deben ser excluidos o la diferencia es NaT, lo cual np.select maneja bien como False por defecto.

condiciones = [
    # C1: Muerte antes de 1 año (Tiempo < 1 año Y ocurrió la muerte)
    (diferencia_tiempo <= un_anio) & es_muerte,
    
    # C2: Muerte entre 1 y 3 años (Tiempo < 3 años Y ocurrió la muerte)
    # No necesitamos la condición > 1 año porque np.select evalúa C1 primero.
    (diferencia_tiempo <= tres_anios) & es_muerte,
    
    # C3: Muerte entre 3 y 5 años (Tiempo < 5 años Y ocurrió la muerte)
    (diferencia_tiempo <= cinco_anios) & es_muerte,
    
    # C4: Sobrevivencia, pero menos de 5 años
    # Paciente VIVO Y el seguimiento es < 5 años (censura temprana)
    (diferencia_tiempo <= cinco_anios) & (df_bi['Vivo_Hoy'] == '1')
]

elecciones = [
    "Muerte al año",          # C1: Murió < 1 año
    "Muerte a los 3 años",    # C2: Murió en [1, 3) años
    "Muerte a los 5 años",    # C3: Murió en [3, 5) años
    "Censurado < 5 años"      # C4: Vivo al control < 5 años
]

# 3. Aplicar np.select
df_bi['Periodo_Sobrevivencia'] = np.select(
    condiciones,
    elecciones,
    # Default: Si ninguna condición anterior se cumple:
    # 1. El paciente está VIVO y el control fue > 5 años. O
    # 2. El paciente MURIÓ y el control fue > 5 años.
    # El caso 2 es "Muerte después de 5 años". El caso 1 es "Sobrevive más de 5 años (Censura)".
    default="Sobrevive más de 5 años"
)
    

C:\Users\SSN0609\AppData\Local\Temp\ipykernel_22532\2440962924.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_bi['Fecha_Control/Muerte'] = pd.to_datetime(df_bi['Fecha_Control/Muerte'], errors='coerce')


In [321]:
# 1. Convertir la diferencia de tiempo (Timedelta) a un número de días (Integer)
df_bi['Tiempo_Supervivencia_Dias'] = diferencia_tiempo.dt.days

# Opcional: Reemplazar valores no válidos (NaT o tiempos negativos) con np.nan
df_bi['Tiempo_Supervivencia_Dias'] = np.where(
    df_bi['Tiempo_Supervivencia_Dias'].isnull() | (df_bi['Tiempo_Supervivencia_Dias'] < 0),
    np.nan,
    df_bi['Tiempo_Supervivencia_Dias']
)

# 2. Crear la variable binaria del Evento (1 = Muerte, 0 = Censura)
# Asumiendo que df_bi['Vivo_Hoy'] es una cadena '0' para Muerte y '1' para Vivo.
df_bi['Evento_Muerte'] = np.where(
    df_bi['Vivo_Hoy'] == '0', # Si el paciente no está vivo (murió)
    1,                       # Evento: 1 (Muerte)
    0                        # Censura: 0 (Vivo / Fin de seguimiento)
)

In [322]:
lst_colum_num=[ 'Sexo_Receptor', 'Edad_Al_Tx', '¿Complicación?',
       'Tiempo_En_Lista_Meses',
       'Año_Tx', 
       'MELD_Score', 'Peso_Pre_Tx', 'BMI',
       'Falla_Cardiaca_Pre_Tx', 'Enfermedad_Coronaria_Pre_Tx',
       'Infarto_Cardiaco_Pre_Tx', 'Diabetes_Mellitus_Pre_Tx',
       'Hipertensión_Arterial_Pre_Tx', 'Antecedente_De_Tabaquismo',
       'Antecedente_De_Alcoholismo', 'LDL_Pre_Tx', 'HDL_Pre_Tx',
       'Triglicéridos_Pre_Tx', 'Colesterol_TotalPre_Tx',
       '#_Unidades_Plaquetas_Tx#1', '#_Unidades_Plasma_Fresco_Congelado_Tx#1',
       '#_Unidades_Glóbulos_Rojos_Tx#1', 'Tiempo_Quirúrgico_Tx#1',
       'Días_Tratamiento_Antibiótico_Previo_A_Tx#1', 'Antibiótico_1__Tx#1',
       'Infección_POP', 'Retrasplante', 'Días_En_UCI',
       'Días_Totales_Intrahospitalarios', 'Sobrevida_PeriQx_30_días_POP',
       'Vivo_Hoy']

for col in df_bi[lst_colum_num]:
    df_bi[col] = df_bi[col].astype(str).str.strip().str.upper()   # quita espacios
    df_bi[col]=df_bi[col].replace(['9999','NAN',"","-","NHD"], '')
    
    df_bi[col]=df_bi[col].replace(['O'], '0')

    df_bi[col] = pd.to_numeric(df_bi[col], errors='coerce')  # Convierte a numérico, reemplaza errores con NaN
    

In [323]:
path=r'C:\Users\SSN0609\OneDrive - Stanley Black & Decker\Sebastian\lina\Data\Processed\Transplantes\transplantes_bi.xlsx'
df_bi.to_excel(path, index=False)